# Working with Environments

By now you've run many experiments in your Azure Machine Learning workspace, and in some cases you've had to specify the particular Python packages required in the environment where the experiment code is run. In this lab, you'll explore environments in a little more detail.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [3]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.10.0 to work with Lab01A


## Prepare Data for an Experiment

In this lab, you'll use a dataset containing details of diabetes patients. Run the cell below to create this dataset (if you created it in a previous lab, the code will find the existing version)

In [4]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


## Create a Training Script

Run the following two cells to create:
1. A folder for a new experiment
2. An training script file that uses **scikit-learn** to train a model and **matplotlib** to plot a ROC curve.

In [5]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

diabetes_training_logistic folder created


In [6]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Overwriting diabetes_training_logistic/diabetes_training.py


## Define an Environment

When you run a Python script as an experiment in Azure Machine Learning, a Conda environment is created to define the execution context for the script. Azure Machine Learning provides a default environment that includes many common packages; including the **azureml-defaults** package that contains the libraries necessary for working with an experiment run, as well as popular packages like **pandas** and **numpy**.

You can also define your own environment and add packages by using **conda** or **pip**, to ensure your experiment has access to all the libraries it requires. 

In [7]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
diabetes_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (conda or pip as required)
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib', 'pandas'],
                                          pip_packages=['azureml-sdk','pyarrow'])

# Add the dependencies to the environment
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

diabetes-experiment-env defined.


Now you can use the environment for the experiment by assigning it to an Estimator (or RunConfig).

The following code assigns the environment you created to a generic estimator, and submits an experiment. As the experiment runs, observe the run details in the widget and in the **azureml_logs/60_control_log.txt** output log, you'll see the conda environment being built.

In [8]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Set the script parameters
script_params = {
    '--regularization': 0.1
}

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input
              script_params=script_params,
              compute_target = 'local',
              environment_definition = diabetes_env,
              entry_script='diabetes_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1596349069_74210eea',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-08-02T06:22:17.898184Z',
 'endTimeUtc': '2020-08-02T06:22:35.669462Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'b84440e7-f495-45cb-b60e-816be54c1e19',
  'azureml.git.repository_uri': 'https://github.com/MicrosoftLearning/DP100',
  'mlflow.source.git.repoURL': 'https://github.com/MicrosoftLearning/DP100',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '17ac2a2b5518c3cfae2d3782d6184374c6c20ed1',
  'mlflow.source.git.commit': '17ac2a2b5518c3cfae2d3782d6184374c6c20ed1',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': '0fea834c-2bb2-482e-bf14-aa3aff3898d9'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'diabetes', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'diabetes_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments':

The experiment successfully used the environment, which included all of the packages it required - you can view the metrics and outputs from the experiment run in Azure Machine Learning Studio, or by running the code below - including the model trained using **scikit-learn** and the ROC chart image generated using **matplotlib**.

In [9]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.7888888888888889
AUC 0.8568469236887952
ROC aml://artifactId/ExperimentRun/dcid.diabetes-training_1596349069_74210eea/ROC_1596349348.png


ROC_1596349348.png
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/9_azureml.log
outputs/diabetes_model.pkl


## Register the Environment

Having gone to the trouble of defining an environment with the packages you need, you can register it in the workspace.

In [10]:
# Register the environment
diabetes_env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200423.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "diabetes-experiment-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "co

Note that the environment is registered with the name you assigned when you first created it (in this case, *diabetes-experiment-env*).

With the environment registered, you can reuse it for any scripts that have the same requirements. For example, let's create a folder and script to train a diabetes model using a different algorithm:

In [11]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

diabetes_training_tree folder created


In [12]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing diabetes_training_tree/diabetes_training.py


Now you can retrieve the registered environment and use it to configure the estimator for a new experiment that runs the alternative training script (there are no script parameters this time because a Decision Tree classifier doesn't require any hyperparameter values).

In [13]:
from azureml.train.estimator import Estimator
from azureml.core import Environment, Experiment
from azureml.widgets import RunDetails

registered_env = Environment.get(ws, 'diabetes-experiment-env')

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input
              compute_target = 'local',
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1596349556_36e186d5',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-08-02T06:25:57.721666Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'c1aee530-d3e9-415e-afdb-d93e6e1c333d',
  'azureml.git.repository_uri': 'https://github.com/MicrosoftLearning/DP100',
  'mlflow.source.git.repoURL': 'https://github.com/MicrosoftLearning/DP100',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '17ac2a2b5518c3cfae2d3782d6184374c6c20ed1',
  'mlflow.source.git.commit': '17ac2a2b5518c3cfae2d3782d6184374c6c20ed1',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': '0fea834c-2bb2-482e-bf14-aa3aff3898d9'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'diabetes', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'diabetes_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'f

This time the experiment runs more quickly because a matching environment has been cached from the previous run, so it doesn't need to be recreated on the local compute. However, even on a different compute target, the same environment would be created and used - ensuring consistency for your experiment script execution context.

Let's look at the metrics and outputs from the experiment.

In [14]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Accuracy 0.8991111111111111
AUC 0.8839290533276825
ROC aml://artifactId/ExperimentRun/dcid.diabetes-training_1596349556_36e186d5/ROC_1596349567.png


ROC_1596349567.png
azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/8_azureml.log
outputs/diabetes_model.pkl


It looks like this model is slightly better than the logistic regression model, so let's register it.

In [15]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator + Environment (Decision Tree)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 6
	 Training context : Estimator + Environment (Decision Tree)
	 AUC : 0.8839290533276825
	 Accuracy : 0.8991111111111111


diabetes_model version: 5
	 Training context : SKLearn Estimator (file dataset)
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889


diabetes_model version: 4
	 Training context : SKLearn Estimator (tabular dataset)
	 AUC : 0.8568632924585982
	 Accuracy : 0.7893333333333333


diabetes_model version: 3
	 Training context : Using Datastore
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889


diabetes_model version: 2
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.8483904671874223
	 Accuracy : 0.7736666666666666


diabetes_model version: 1
	 Training context : Estimator
	 AUC : 0.8483377282451863
	 Accuracy : 0.774


amlstudio-lab02b-predict-diabe version: 1
	 CreatedByAMLStudio : true




## View Registered Environments

In addition to registering your own environments, you can leverage pre-built "curated" environments for common experiment types. The following code lists all registered environments:

In [16]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

Name diabetes-experiment-env
Name AzureML-Tutorial
Name AzureML-Minimal
Name AzureML-Chainer-5.1.0-GPU
Name AzureML-PyTorch-1.2-CPU
Name AzureML-TensorFlow-1.12-CPU
Name AzureML-TensorFlow-1.13-CPU
Name AzureML-PyTorch-1.1-CPU
Name AzureML-TensorFlow-1.10-CPU
Name AzureML-PyTorch-1.0-GPU
Name AzureML-TensorFlow-1.12-GPU
Name AzureML-TensorFlow-1.13-GPU
Name AzureML-Chainer-5.1.0-CPU
Name AzureML-PyTorch-1.0-CPU
Name AzureML-Scikit-learn-0.20.3
Name AzureML-PyTorch-1.2-GPU
Name AzureML-PyTorch-1.1-GPU
Name AzureML-TensorFlow-1.10-GPU
Name AzureML-PyTorch-1.3-GPU
Name AzureML-TensorFlow-2.0-CPU
Name AzureML-PyTorch-1.3-CPU
Name AzureML-TensorFlow-2.0-GPU
Name AzureML-PySpark-MmlSpark-0.15
Name AzureML-AutoML
Name AzureML-PyTorch-1.4-GPU
Name AzureML-PyTorch-1.4-CPU
Name AzureML-VowpalWabbit-8.8.0
Name AzureML-Hyperdrive-ForecastDNN
Name AzureML-AutoML-GPU
Name AzureML-AutoML-DNN-GPU
Name AzureML-AutoML-DNN
Name AzureML-Designer-R
Name AzureML-Designer-Recommender
Name AzureML-Designer-Tr

All curated environments have names that begin ***AzureML-*** (you can't use this prefix for your own environments).

Let's explore the curated environments in more depth and see what packages are included in each of them.

In [17]:
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-Tutorial
packages channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.10.0
  - azureml-defaults==1.10.0
  - azureml-telemetry==1.10.0
  - azureml-train-restclients-hyperdrive==1.10.0
  - azureml-train-core==1.10.0
  - azureml-widgets==1.10.0
  - azureml-pipeline-core==1.10.0
  - azureml-pipeline-steps==1.10.0
  - azureml-opendatasets==1.10.0
  - azureml-automl-core==1.10.0.post1
  - azureml-automl-runtime==1.10.0.post1
  - azureml-train-automl-client==1.10.0
  - azureml-train-automl-runtime==1.10.0
  - azureml-train-automl==1.10.0
  - azureml-train==1.10.0
  - azureml-sdk==1.10.0
  - azureml-interpret==1.10.0
  - azureml-tensorboard==1.10.0
  - azureml-mlflow==1.10.0
  - mlflow
  - sklearn-pandas
- pandas
- numpy
- tqdm
- scikit-learn
- matplotlib
name: azureml_46fb7d2fe1381cf4c90841fb72b9a774

Name AzureML-Minimal
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.10.0
  - azureml-defaults==1.10.

> **More Information**: For more information about environments in Azure Machine Learning, see [the Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)